In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
import re

checked /robots.txt on ryman.com, for user-agent * did not see /events in disallow, so I think I'm good

In [ ]:
URL = 'https://ryman.com/events/'

ryman = requests.get(URL)

In [ ]:
ryman

In [ ]:
type(ryman)

In [ ]:
ryman.status_code

In [ ]:
#in class he did .content instead of .text (unicode vs byte).text gives you the response in Unicode, .content gives it to you in bytes, more on what any of that means here, if you wish to delve deep:
# https://betterprogramming.pub/strings-unicode-and-bytes-in-python-3-everything-you-always-wanted-to-know-27dc02ff2686

soup = BS(ryman.text)

In [ ]:
soup

In [ ]:
# because h2 is too far up to directly get title, trying a lower structure, a. 
# but because there are so many other a tags, need to specify class
header = soup.findAll('a', attrs = {'class': 'tribe-event-url'})

In [ ]:
len(soup.findAll('a', attrs = {'class': 'tribe-event-url'}))

In [ ]:
# now title is much higher up in structure, than when i grabbed by h2
header

In [ ]:
first_header = header[0]
print(type(first_header))
first_header

In [ ]:
first_header['title']

In [ ]:
header_title = [x.get('title') for x in header]

In [ ]:
header_title

In [ ]:
date_time = soup.findAll('time')
print(type(date_time))
date_time

In [ ]:
#matches length for title tags which should be good?
len(date_time)

In [ ]:
first_date_time = date_time[0]
first_date_time

In [ ]:
first_date_time['datetime']

In [ ]:
#could have done x.text instead of doing datetime
date_time_ext = [x.get('datetime') for x in date_time]

In [ ]:
date_time_ext

In [ ]:
type(date_time_ext)

In [ ]:
list_tup = list(zip(header_title, date_time_ext))

In [ ]:
list_tup

In [ ]:
ryman_shows = pd.DataFrame(list_tup, columns=['headliner', 'date_time'])

In [ ]:
ryman_shows.head()

In [ ]:
ryman_shows['date'] = ryman_shows['date_time'].str[:10]

In [ ]:
 ryman_shows['time'] = ryman_shows['date_time'].str[10:]

In [ ]:
ryman_shows.head()

In [ ]:
# comment out here to bring back date_time combined
ryman_shows = ryman_shows.drop('date_time', axis=1)

In [ ]:
ryman_shows

## attempt at #4, applying scrape to multiple pages

### another helpful scrape resource - https://www.learndatasci.com/tutorials/ultimate-guide-web-scraping-w-python-requests-and-beautifulsoup/

In [ ]:
# got this from - https://stackoverflow.com/questions/69546438/how-to-create-a-for-loop-when-scraping-multiple-pages-of-a-url
pageCount = 6
urls_list = []
base = 'https://ryman.com/events/list/?tribe_event_display=list&tribe_paged={}'

for x in range(pageCount)[1:] :
    urls_list.append(base.format(x))

In [ ]:
urls_list

In [ ]:
ryman_5 = [requests.get(x) for x in urls_list]

In [ ]:
ryman_5

In [ ]:
soup5 = [BS(x.text) for x in ryman_5]

In [ ]:
soup5

In [ ]:
headerall5 = [x.findAll('a', attrs = {'class': 'tribe-event-url'}) for x in soup5]

In [ ]:
len(headerall5)

In [ ]:
#remember this is the headers for all 5 pages, but it is broken out
# in a list of headers for each page as each value of list(as shown in 
# the above cell), so need to keep using loops to move through the lists
headerall5

In [ ]:
# so this wont work because .get wont work in a list, is there a way around this? fixed in below cell
#header5_title = [x.get('title') for x in headerall5]

In [ ]:
#.append will create list of lists for each webpage, .extend will make single list containing all webpages

# in words, for each webpage in headerall5 list, make a list in header_list(new empty list) of results from x.get 
# webpage is the chosen variable name of headerall5 list, meaning each of the 5 webpages, and x is the variable of 
# each webpage, meaning the 20 'titles' per webpage.
header_list = []

for webpage in headerall5: 
    header_list.extend([x.get('title') for x in webpage])
    
header_list

In [ ]:
len(header_list)

## get datetime for all 5 pages

In [ ]:
dtall5 = [x.findAll('time') for x in soup5]

In [ ]:
dtall5

In [ ]:
dt_list = []

for page in dtall5:
    dt_list.extend([x.get('datetime') for x in page])
    
dt_list

In [ ]:
len(dt_list)

In [ ]:
rymanall_tup = list(zip(header_list, dt_list))

In [ ]:
ryman5_shows = pd.DataFrame(rymanall_tup, columns=['headliner', 'date_time'])

In [ ]:
ryman5_shows

In [ ]:
ryman5_shows['date'] = ryman5_shows['date_time'].str[:10]

In [ ]:
 ryman5_shows['time'] = ryman5_shows['date_time'].str[10:]

In [ ]:
ryman5_shows = ryman5_shows.drop('date_time', axis=1)

In [ ]:
ryman5_shows.head()

In [ ]:
ryman5_shows.dtypes

In [ ]:
ryman5_shows['date'] = pd.to_datetime(ryman5_shows['date'])

In [ ]:
#to get nicer looking dates
ryman5_shows['date'] = ryman5_shows['date'].dt.strftime('%A,%d,%B,%Y')

In [ ]:
ryman5_shows

# Bonus 1 add opener

In [ ]:
opener_all5 = [x.findAll('span', attrs = {'class': 'opener'}) for x in soup5]

In [ ]:
opener_all5

## before I try to get 5 pages of openers, I want to look at how to pull just one page

In [ ]:
opener_1 = soup.findAll('span', attrs = {'class': 'opener'})

In [ ]:
opener_1first = opener_1[0]

In [ ]:
opener_1first.text

In [ ]:
opener_1text = [x.text for x in opener_1]

In [ ]:
opener_1text

In [ ]:
# this should only be 20, why are there extra openers?! after looking, some have 2 openers, 
# and some have no openers...
len(opener_1text)

In [ ]:
#could search above span, then do a loop to find only the first opener per span?
opener_1up = soup.findAll('div', attrs = {'class': 'tribe-beside-image'})

In [ ]:
opener_1up

In [ ]:
opener_1up_name = [x.find('span', attrs = {'class':'opener'}) for x in opener_1up]

In [ ]:
opener_1up_name

In [ ]:
# so now this only has 20, I think this got rid of the ones with 2 openers, 
# but Im not sure if there are any with no openers that may still be an issue?
len(opener_1up_name)

In [ ]:
# because some are empty, i cant get the text for these so i can try using regex instead to 
# pick out the text i want in the next cell
#opener_1up_text = [x.text for x in opener_1up_name]

In [ ]:
#lets start by looking at just one at a time
opener_1up_name_first = opener_1up_name[0]

In [ ]:
opener_1up_name_first

In [ ]:
# i dont think this dtype works for regex
type(opener_1up_name_first)

In [ ]:
# need to convert this to string
opener_1up_name

In [ ]:
#this converts it to string
opener_1up_string = []
for x in opener_1up_name :
    opener_1up_string.append(str(x))

In [ ]:
opener_1up_string

In [ ]:
type(opener_1up_string)

In [ ]:
opener_string_1 = opener_1up_string[0]

In [ ]:
opener_string_1

In [ ]:
# now regex works!
re.search('opener', opener_string_1)

In [ ]:
#nope
re.search('[\"opener\">][a*\w][</span>]', opener_string_1)

In [ ]:
#nope
re.search('opener\">\w\a*', opener_string_1)

In [ ]:
# got it!
re.search('>.*<', opener_string_1)

In [ ]:
re.search('>.*<', opener_string_1).group(0)

In [ ]:
type(re.search('>.*<', opener_string_1).group(0))

In [ ]:
#as you can see in the below cell this is not what i wanted, and this wont work with .group(0)
# for some reason, so I still dont know how to get this to output matching text instead of match object
opener_text = []
for x in opener_1up_string :
    opener_text.append(re.search('>(.*)<', x))

In [ ]:
opener_text[0].group(1)

In [ ]:
# so with an if/else if oject type is No text (or whatever its called) print none
# and if text is less than 4 print none
for x in opener_text:
    if x is not None:
        print(len(x.group(0)))
    else :
        print(0)

In [ ]:
#check around cell 64 in class recording for a fix to this 
for x in opener_text:
    if x is not None:
        for x in opener_1up_string :
            opener_text.append(re.search('>(.*)<', x).group(1))
    else :
        print(0)